## TOOL UPDATED: Dec 20 2022, Henrik Loecke

<!-- 

To run this notebook, click menu Cell -> Run All

User input has been moved away from this notebook so it can easily be replaced by new versions.
 
Please open Calibration_Variables.py, in the same folder as this notebook, to edit user input there.

All varables with path must start with 'r', e.g. r'C:\Projects'

It must contain the following variables:

model_area:                          Short area name like 'VSA' or LISA'
generate_confidence_csvs:            Generate csv files for confidence maps, True/False
map_point_spacing:                   Space between dots in confidence maps, e.g. 100 (in meter but number withouth unit)
use_accumulation:                    Use for models with proper tree structure, all models except VSA, True/False
slope_source_unit_meter_per_meter    This is the case for NSSA and FSA, in VSA it is per thousand
model_area_strict_match:             If True, accept 'VSA' but not 'VSA-2019'. If False, accept both.
output_folder:                       Folder path where reports are to be created.
result_folder:                       Folder path of result files.
calibration_sheet:                   Full path, folder included, of calibration parameter sheet.
model:                               Full path, folder included, of model database.
summation_csv:                       Full path, folder included, of summation.csv.
node_csv:                            Full path, folder included, of MH_Zones.csv.
outfall_csv:                         Full path, folder included, of Outfall_Summary.csv.
rainfall_dfs0_file:                  Full path, folder included, of rainfall dfs0 file.
map_folder:                          Folder path where report maps are located.
dfs0_folders:                        Python list of all folders holding measurement dfs0 files.
 -->


In [301]:
#PERMANENT CELL 1

import os
import re
import mikeio
import mikeio1d
from mikeio1d.res1d import Res1D
from mikeio.dfs0 import Dfs0
import numpy as np
import pandas as pd
import datetime as dt
import pickle
import plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from IPython.display import clear_output
import math
import sqlite3


In [303]:
#PERMANENT CELL 2
#The external script Read_Parameters.py is called to read data from the model.

import subprocess


parameter_script = r"Read_Parameters.py"
bat_file_path = 'Read_Parameters.bat'
bat_file = open(bat_file_path, "w")
bat_file.write('"' + parameter_script + '" "' + os.getcwd() + '" "' + model + '" ' + str(use_accumulation))
bat_file.close()
subprocess.call([bat_file_path]) 




0

In [326]:
#PERMANENT CELL 3
#

dwf_csv = r"DWF_Specs.csv"
res_types = ['Mixed','ResHD','ResLD']
ici_types = ['Commercial','Industrial','Institutional']
ww_types = res_types + ici_types

all_types = []
for res_type in res_types:
    all_types.append([res_type,'Population','pe','m3/pe/d'])
for ici_type in ici_types:
    all_types.append([ici_type,'Area','ha','m3/ha/d'])
all_types.append(['Baseflow','','',''])
all_types.append(['Total','','',''])
all_types = pd.DataFrame(all_types, columns =['Load_Type','Description','Unit1','Unit2'])
all_types.set_index('Load_Type',inplace=True)

period_specs = []
period_specs.append('DWF')
for i in range(1,6):
    period_specs.append('WWF' + str(i))
period_specs

dwf_specs = pd.read_csv(os.getcwd() + '\\' + dwf_csv)
dwf_specs.set_index('Zone',inplace=True)

In [333]:
#PERMANENT CELL 4

wwf_csv = r"WWF_Specs.csv"

#used to maintain consistent color order for overflows
colors = ['black','purple','green','orange','grey','brown']

gauges = pd.read_excel(calibration_sheet,sheet_name="Gauges")
gauges = gauges.loc[:, :'Shift Y (m)']
if model_area_strict_match == True:
    gauges = gauges[gauges.Model==model_area]
else:
    gauges = gauges[gauges.Model.str.contains(model_area)]
gauges.set_index('Gauge',inplace=True)


periods = pd.read_excel(calibration_sheet,sheet_name="Periods")
if model_area_strict_match == True:
    periods = periods[periods.Model==model_area]
else:
    periods = periods[periods.Model.str.contains(model_area)]

summation_df = pd.read_csv(summation_csv,dtype={'MUID': str,'SUMTO': str})

report_text = pd.read_excel(calibration_sheet,sheet_name="Report_Text")
report_text = report_text[report_text.Model.notna()]
if model_area_strict_match == True:
    report_text = report_text[report_text.Model==model_area]
else:
    report_text = report_text[report_text.Model.str.contains(model_area)]
report_text.fillna('This section was left blank.', inplace=True)

wwf_specs = pd.read_csv(wwf_csv)
wwf_specs.set_index('Location',inplace=True)
wwf_stats_specs = pd.read_csv('WWF_Stats_Specs.csv')


if generate_confidence_csvs:
    
    map_periods = pd.read_excel(calibration_sheet,sheet_name="Periods_Map")
    map_periods = map_periods.loc[:, :'Model']
    if model_area_strict_match == True:
        map_periods = map_periods[map_periods.Model==model_area]
    else:
        map_periods = map_periods[map_periods.Model.str.contains(model_area)]

    thresholds = pd.read_excel(calibration_sheet,sheet_name="Thresholds",skiprows=1)
    thresholds.rename(columns={'Unnamed: 0':'Color'},inplace=True)
    for column in thresholds.columns[1:]:
        thresholds.rename(columns={column:column + ' ' + thresholds.loc[0,column]},inplace=True)
    for column in thresholds.columns[1:]:
        thresholds.rename(columns={column:column.replace('.1','')},inplace=True)
        thresholds.rename(columns={column:column.replace('.2','')},inplace=True)
    thresholds.drop(index=0,inplace=True)
    thresholds.drop(thresholds.index[3:],inplace=True) 
    thresholds.sort_values(by='DWF Flow Status',ascending=False,inplace=True)
    thresholds.reset_index(inplace=True)

    map_periods_check = map_periods.loc[:, :'Meter Status']
    map_periods_check_count = map_periods_check[map_periods_check['Meter Status']=='Primary']\
        [['Zone','Meter Status']].groupby(['Zone']).count()
    map_periods_check_count.rename(columns={'Meter Status':'Primary Count'},inplace=True)
    map_periods_check = pd.merge(map_periods_check,map_periods_check_count,on=['Zone'],how='left')
    multiple_primary = list(map_periods_check_count[map_periods_check_count['Primary Count']>1].index)

    missing_primary = []
    for zone in list(gauges.Location.unique()):
        if not zone in (map_periods_check_count.index):
            missing_primary.append(zone)

    if len(multiple_primary) > 0 or len(missing_primary) > 0 :
        error_message = 'Please correct the following errors in ' + os.path.basename(calibration_sheet) + ', sheet Periods_Map.\n\n'
        error_message += 'The following zones have multiple primary meters: '  + ','.join(multiple_primary) + '.\n'
        error_message += 'The following zones have no primary meters: '  + ','.join(missing_primary) + '.\n'
        raise ValueError(error_message)


In [336]:
#PERMANENT CELL 5
#Import manhole spilling from ADDOUT.res1d files.

node_zones = pd.read_csv(node_csv,dtype={'Node': str})
node_zones.set_index(node_zones.Node,inplace=True)
spill_zones = list(node_zones.Zone.unique())

first_round = True
for f in os.listdir(result_folder):
    if '.ADDOUT.res1d' in f:
        print(f)
        res1d = Res1D(result_folder + '\\' + f)
        flood_types = ['WaterFlowRateAboveGround','WaterSpillDischarge']
        cover_types = ['Normal','Spilling']

        first_round_in_result = True
        res_zones = set()
        for node in res1d.data.Nodes:
            muid = node.Id
            for i, flood_type in enumerate(flood_types):
                ts = res1d.query.GetNodeValues(muid,flood_type)
                col_name = muid + '-' + cover_types[i]
                if ts != None:
                    if max(ts) > 0 or first_round_in_result == True:
                        spill_df = pd.DataFrame(index=res1d.time_index)
                        spill_df['Node'] = muid
                        zone = node_zones.loc[muid,'Zone']
                        spill_df['Zone'] = zone
                        spill_df['Spill'] = ts
                        spill_df['Spill'] = spill_df['Spill'] * 1000
                        
                        res_zones.add(zone)
                        
                        if first_round == True:
                            spill_df_all = spill_df.copy()
                        else:
                            spill_df_all = pd.concat([spill_df_all,spill_df])                                                        
                        first_round = False
                        first_round_in_result = False
                        
        #Create empty df for any zone not represented to show 0 on the graph.
        for spill_zone in spill_zones:
            if not spill_zone in res_zones:
                spill_df = pd.DataFrame(index=res1d.time_index)
                spill_df['Node'] = 'X'
                spill_df['Zone'] = spill_zone
                spill_df['Spill'] = 0           
                spill_df_all = pd.concat([spill_df_all,spill_df])                                                        
                                                                                      
spill_df_zones = spill_df_all.copy()
spill_df_zones['Date_Time'] = spill_df_zones.index
spill_df_zones.drop(columns=['Node'],inplace=True)
spill_df_zones = spill_df_zones.groupby(['Zone','Date_Time']).sum()
spill_df_zones.reset_index(inplace=True)

                        


FSA_DWF_2021-07-28_5d_2021pop_Base.ADDOUT.res1d
FSA_DWF_2021-09-01_5d_2021pop_Base.ADDOUT.res1d
FSA_DWF_2021-09-21_5d_2021pop_Base.ADDOUT.res1d
FSA_WWF_2021-11-12_5d_2020pop_Base.ADDOUT.res1d
FSA_WWF_2021-11-24_8d_2020pop_Base.ADDOUT.res1d


In [315]:
#PERMANENT CELL 6
#List all network elements to be imported.

nodes1 = gauges[gauges['Level Type']=='Node']['Node1 (Or Pipe if pipe level)'].dropna().unique().flatten().tolist()
nodes2 = gauges[gauges['Level Type']=='Node']['Node2'].dropna().unique().flatten().tolist()
level_nodes = nodes1 + nodes2
#Convert all to string
level_nodes = [str(x) for x in level_nodes]
#Make set to clear duplicates
level_nodes = set(level_nodes)

ds_level_pipes = gauges[gauges['Level Type']=='Link(DS)']['Node1 (Or Pipe if pipe level)'].dropna().unique().flatten().tolist()
us_level_pipes = gauges[gauges['Level Type']=='Link(US)']['Node1 (Or Pipe if pipe level)'].dropna().unique().flatten().tolist()
flow_pipes = gauges.Pipe.dropna().unique().flatten().tolist()

#Convert all to string
ds_level_pipes = [str(x) for x in ds_level_pipes]
us_level_pipes = [str(x) for x in us_level_pipes]
flow_pipes = [str(x) for x in flow_pipes]

#Add outfalls
outfalls = pd.read_csv(outfall_csv,dtype={'Weir': str,'Outfall': str})
outfalls['Res_ID'] = ''
for index, row in outfalls.iterrows():
    prefix = ''
    
    if row['Layer'] == 'Summation':
        prefix = ''
    elif row['Layer'] != 'msm_Link':
        prefix = row['Layer'][4:] + ':'
    muid = prefix + row['Weir']
    
    if row['Layer'] != 'Summation':
        outfalls.iloc[index,3] = row['Layer'][4:]
    outfalls.iloc[index,4] = muid
     
    print(muid)
    if row['Layer'] != 'Summation':
        flow_pipes.append(muid)
        
        
for index, row in summation_df.iterrows():
    prefix = ''
    
    if row['Layer'] != 'msm_Link':
        prefix = row['Layer'][4:] + ':'
    muid = prefix + row['MUID']
    summation_df.iloc[index,0] = muid
    
    flow_pipes.append(muid)
    
    
#Make set to clear duplicates
ds_level_pipes = set(ds_level_pipes)
us_level_pipes = set(us_level_pipes)
flow_pipes = set(flow_pipes)


Weir:11E
Weir:13W
Weir:15W
Weir:16W_New
Weir:17W
Weir:1F
Weir:1G
Weir:1W
Weir:20th Street PS Overflow
Weir:2F
Weir:2G
Weir:2S
Weir:3F
Weir:3S
Weir:3W
Weir:4F
Weir:4S
Weir:52121 Port Coquitlam Weir
Weir:5BE
Weir:6E
Weir:8AW
Weir:Baynes Overflow
Orifice:Braid Street Gate
Weir:Cliff Ave N Intercept
Weir:Cliff Ave N Wet Well Overflow
Orifice:KatzieDuckBill
link:Link_156_FSA_S
link:Link_177_FSA_S
Weir:marshend
Weir:Maryhill Overflow Weir
Weir:NS4-SSO
Weir:NW CSO Tank Overflow Gate
Orifice:Orifice_1
Weir:ROYAL AVEPS OUTFALL
Weir:Short St
Weir:Slaughterhouse
Weir:SSO weir
Valve:Valve_2


In [331]:
#PERMANENT CELL 7
#Import dfs0 files

#Import rain gauges
rainfall = mikeio.read(rainfall_dfs0_file).to_dataframe()
warnings = []

#Import flow/level gauges
gauge_ids = []
first_dfs0 = True
for dfs0_folder in dfs0_folders:
    for f in os.listdir(dfs0_folder):
        if f[-5:]=='.dfs0':
            
            print(f)
            if 'ps' in f.lower():
                gauge_id = re.split(r'[.]',f)[0]
            else:
                gauge_id = re.split(r'_|[.]',f)[0]
            res = mikeio.read(dfs0_folder + '\\' + f)
            ts = res.to_dataframe()

            gauge_ids.append(gauge_id)
            
            first_level = True
            second_level = True
            first_velocity = True
            for i, column in enumerate(ts.columns):
                if i == 0:
                    ts.rename(columns={ts.columns[0]:'Flow'},inplace=True)
                    if 'meter pow 3 per sec' in str(res.items[0]):
                        ts.Flow = ts.Flow * 1000
                    elif 'liter per sec' in str(res.items[0]):
                        ts.Flow = ts.Flow 
                    else:
                        warnings.append('First item in ' + f + ' does not appear to be type Discharge')
                        
                elif str(res.items[i])[-21:] == '<Water Level> (meter)': 
                    if first_level == True:
                        ts.rename(columns={ts.columns[i]:'Level'},inplace=True)
                        first_level = False
                    elif second_level == True:
                        ts.rename(columns={ts.columns[i]:'Level2'},inplace=True)
                        second_level = False
                    

                elif str(res.items[i])[-31:] == '<Flow velocity> (meter per sec)' and first_velocity == True:
                    ts.rename(columns={ts.columns[i]:'Velocity'},inplace=True)
                    first_velocity = False
            
            if not 'Flow' in ts.columns:
                ts['Flow'] = np.nan                     
            if not 'Velocity' in ts.columns:
                ts['Velocity'] = np.nan
            if not 'Level' in ts.columns:
                ts['Level'] = np.nan
            if not 'Level2' in ts.columns:
                ts['Level2'] = np.nan
            
            ts['Gauge'] = gauge_id
            ts = ts[['Gauge','Flow','Level','Level2','Velocity']]
            ts['Seconds'] = ts.index.to_series().diff().astype('timedelta64[s]').fillna(method='bfill')
            ts['Volume'] = ts.Flow * ts.Seconds / 1000
            if first_dfs0 == True:
                measured = ts.copy()
            else:
                measured = pd.concat([measured,ts])
            first_dfs0 = False
    
for warning in warnings:
    print (warning)
    


        

AL1_20210601_20211006_5min.dfs0
Went here
AL3_20210921_20220401_5min.dfs0
Went here
BI1_20210101_20221017_5min.dfs0
Went here
BIN18_20211123_20220401_5min.dfs0
Went here
BN21_20211022_20220401_5min.dfs0
Went here
BNC9_20211124_20220401_5min.dfs0
Went here
BNP17_20210904_20220401_5min.dfs0
Went here
CQ1_20210101_20221017_5min.dfs0
Went here
CQ20_20210915_20220401_5min.dfs0
Went here
CV14_20211022_20220401_5min.dfs0
Went here
FST6_20210924_20220401_5min.dfs0
Went here
GLT7A_20211008_20220401_5min.dfs0
Went here
HT4_20210814_20220401_5min.dfs0
Went here
LKI55_20210903_20220401_5min.dfs0
Went here
LZ20_20210903_20220401_5min.dfs0
Went here
LZ4_20210101_20221017_5min.dfs0
Went here
LZG32_20210904_20220401_5min.dfs0
Went here
MH20_20210915_20220401_5min.dfs0
Went here
ML1_20210101_20221017_5min.dfs0
Went here
ML2_20210101_20221017_5min.dfs0
Went here
ML7_20210814_20220401_5min.dfs0
Went here
NC1_20210101_20221017_5min.dfs0
Went here
NR4B_20210814_20220401_5min.dfs0
Went here
NSM11_20211007_2

In [335]:
#PERMANENT CELL 8
#Import network result files

results = []
first_level = True
first_flow = True
first_velocity = True

for f in os.listdir(result_folder):
    if f[-6:]=='.res1d' and not 'ADDOUT' in f and not 'RR' in f and not 'UserSpecified' in f and not 'hotstart' in f.lower():
        res1d = Res1D(result_folder + '\\' + f)
        reaches = res1d.data.Reaches
        nodes = res1d.data.Nodes
        
        print ("Importing network " + f + " at " + str(dt.datetime.now()))

        for i, node in enumerate(nodes):
            
            muid = node.Id
            if muid in level_nodes:
                print ("Importing node " + node.Id + " at " + str(dt.datetime.now()))
                
                level_df = pd.DataFrame(index = res1d.time_index)
                level_df['ResultFile'] = f
                level_df['MUID'] = muid
                level_df['Level'] = res1d.query.GetNodeValues(muid, "WaterLevel")                          

                level_df['DateTimeRef'] = level_df.index

                if first_level == True:
                    level_df_all = level_df.copy()
                else:
                    level_df_all = pd.concat([level_df_all,level_df])
                first_level = False
        
        
        first_round = True
        for i, reach in enumerate(reaches):

            muid = reach.Id[:reach.Id.rfind('-')]

            if muid in us_level_pipes or muid in ds_level_pipes:
            
                print ("Importing pipe " + reach.Id + " level at " + str(dt.datetime.now()))


                if muid in us_level_pipes: 
                    values = res1d.query.GetReachStartValues(muid, "WaterLevel")
                else:
                    values = res1d.query.GetReachEndValues(muid, "WaterLevel")

                level_df = pd.DataFrame(index = res1d.time_index)
                level_df['ResultFile'] = f
                level_df['MUID'] = muid
                level_df['Level'] = values                          

                level_df['DateTimeRef'] = level_df.index

                if first_level == True:
                    level_df_all = level_df.copy()

                else:
                    level_df_all = pd.concat([level_df_all,level_df])
                first_level = False
                        
                        
            
        for i, reach in enumerate(reaches):
            
            muid = reach.Id[:reach.Id.rfind('-')]

            if muid in flow_pipes or muid in list(summation_df.MUID):
                print ("Importing pipe " + reach.Id + " discharge at " + str(dt.datetime.now()))

                values = res1d.query.GetReachEndValues(muid, "Discharge")
                flow_df = pd.DataFrame(index = res1d.time_index)
                flow_df['ResultFile'] = f
                flow_df['MUID'] = muid
                flow_df['DateTimeRef'] = flow_df.index
                flow_df['Discharge'] = values                          
                flow_df['Discharge'] = flow_df['Discharge'] * 1000

                flow_df['Seconds'] = flow_df.index.to_series().diff().astype('timedelta64[s]').fillna(method='bfill')
                flow_df['Volume'] = flow_df.Discharge * flow_df.Seconds / 1000
                flow_df.drop(columns=['Seconds'],inplace=True)

                if first_flow == True:
                    flow_df_all = flow_df.copy()

                else:
                    flow_df_all = pd.concat([flow_df_all,flow_df])
                first_flow = False
                        
        for i, reach in enumerate(reaches):
            
            muid = reach.Id[:(-1 * len(str(i)) - 1)]

            if muid in flow_pipes:
                print ("Importing pipe " + reach.Id + " velocity at " + str(dt.datetime.now()))

                values = res1d.query.GetReachEndValues(muid, "FlowVelocity")
                velocity_df = pd.DataFrame(index = res1d.time_index)
                velocity_df['ResultFile'] = f
                velocity_df['MUID'] = muid
                velocity_df['DateTimeRef'] = velocity_df.index
                velocity_df['Velocity'] = values                          


                if first_velocity == True:
                    velocity_df_all = velocity_df.copy()
                else:
                    velocity_df_all = pd.concat([velocity_df_all,velocity_df])
                first_velocity = False

        first_round = False
        
#Summation
df_result_sum = pd.merge(flow_df_all,summation_df,how='inner',on=['MUID'])
df_result_sum = df_result_sum.groupby(['ResultFile','SUMTO','DateTimeRef']).agg({'Discharge':'sum','Volume':'sum'})
df_result_sum.reset_index(inplace=True)
df_result_sum.set_index('DateTimeRef',drop=False,inplace=True)
df_result_sum.rename(columns = {'SUMTO':'MUID'},inplace=True)

flow_df_all = pd.concat([flow_df_all,df_result_sum])


Importing network FSA_DWF_2021-07-28_5d_2021pop_Base.res1d at 2022-12-19 14:07:12.778005
Importing node 10017 at 2022-12-19 14:07:12.778005
Importing node 1021 at 2022-12-19 14:07:12.789039
Importing node 10391 at 2022-12-19 14:07:12.793073
Importing node 1047 at 2022-12-19 14:07:12.797075
Importing node 1071 at 2022-12-19 14:07:12.800078
Importing node 1104 at 2022-12-19 14:07:12.804107
Importing node 1119 at 2022-12-19 14:07:12.807110
Importing node 1152 at 2022-12-19 14:07:12.811113
Importing node 20thSt_PS_WetWell at 2022-12-19 14:07:12.821146
Importing node 22338 at 2022-12-19 14:07:12.824175
Importing node 22357 at 2022-12-19 14:07:12.828179
Importing node 23675 at 2022-12-19 14:07:12.833225
Importing node 24694 at 2022-12-19 14:07:12.837228
Importing node 24802 at 2022-12-19 14:07:12.841232
Importing node 4201 at 2022-12-19 14:07:12.848262
Importing node 4635 at 2022-12-19 14:07:12.852294
Importing node 4666 at 2022-12-19 14:07:12.856296
Importing node 6901 at 2022-12-19 14:07:1

Importing pipe Weir:SSO weir-7270 discharge at 2022-12-19 14:07:13.626619
Importing pipe Valve:Valve_2-7278 discharge at 2022-12-19 14:07:13.633626
Importing pipe Orifice:Braid Street Gate-7398 discharge at 2022-12-19 14:07:13.640632
Importing pipe Orifice:KatzieDuckBill-7403 discharge at 2022-12-19 14:07:13.647638
Importing pipe Orifice:Orifice_1-7408 discharge at 2022-12-19 14:07:13.654645
Importing pipe 40079-341 velocity at 2022-12-19 14:07:13.661651
Importing pipe 40178-431 velocity at 2022-12-19 14:07:13.667656
Importing pipe 40494-610 velocity at 2022-12-19 14:07:13.671660
Importing pipe 40503-618 velocity at 2022-12-19 14:07:13.674663
Importing pipe 40934-659 velocity at 2022-12-19 14:07:13.678667
Importing pipe 40995-703 velocity at 2022-12-19 14:07:13.681669
Importing pipe 41129-764 velocity at 2022-12-19 14:07:13.685697
Importing pipe 41181-802 velocity at 2022-12-19 14:07:13.689701
Importing pipe 41630-1002 velocity at 2022-12-19 14:07:13.693731
Importing pipe 41638-1005 ve

Importing node 8814 at 2022-12-19 14:07:19.198567
Importing node 89546 at 2022-12-19 14:07:19.203572
Importing node 89574 at 2022-12-19 14:07:19.209577
Importing node 9826 at 2022-12-19 14:07:19.214582
Importing node 9960 at 2022-12-19 14:07:19.220587
Importing node BI1_Dummy at 2022-12-19 14:07:19.225592
Importing node FSA_North_Outlet2 at 2022-12-19 14:07:19.230596
Importing node Node_5550 at 2022-12-19 14:07:19.236602
Importing node Node_6411 at 2022-12-19 14:07:19.241606
Importing node NW_20th at 2022-12-19 14:07:19.247612
Importing pipe 40079-341 discharge at 2022-12-19 14:07:19.279722
Importing pipe 40178-431 discharge at 2022-12-19 14:07:19.287758
Importing pipe 40494-610 discharge at 2022-12-19 14:07:19.295793
Importing pipe 40503-618 discharge at 2022-12-19 14:07:19.302824
Importing pipe 40934-659 discharge at 2022-12-19 14:07:19.309829
Importing pipe 40995-703 discharge at 2022-12-19 14:07:19.316865
Importing pipe 41129-764 discharge at 2022-12-19 14:07:19.324901
Importing pi

Importing pipe 52458-5640 velocity at 2022-12-19 14:07:20.284051
Importing pipe 52588-5748 velocity at 2022-12-19 14:07:20.290056
Importing pipe 52670-5813 velocity at 2022-12-19 14:07:20.296062
Importing pipe 52671-5814 velocity at 2022-12-19 14:07:20.301066
Importing pipe 52712-5849 velocity at 2022-12-19 14:07:20.307072
Importing pipe 52713-5850 velocity at 2022-12-19 14:07:20.313077
Importing pipe 52723-5857 velocity at 2022-12-19 14:07:20.318082
Importing pipe 53089-6110 velocity at 2022-12-19 14:07:20.324087
Importing pipe 53261-6239 velocity at 2022-12-19 14:07:20.330093
Importing pipe 53290-6268 velocity at 2022-12-19 14:07:20.336127
Importing pipe 53405-6367 velocity at 2022-12-19 14:07:20.341131
Importing pipe 53414-6377 velocity at 2022-12-19 14:07:20.347177
Importing pipe 53540-6484 velocity at 2022-12-19 14:07:20.352219
Importing pipe 53664-6541 velocity at 2022-12-19 14:07:20.358223
Importing pipe 53713-6581 velocity at 2022-12-19 14:07:20.364256
Importing pipe 53917-6597

Importing pipe 51099-4891 discharge at 2022-12-19 14:07:26.405323
Importing pipe 52010-5292 discharge at 2022-12-19 14:07:26.418335
Importing pipe 52073-5341 discharge at 2022-12-19 14:07:26.427343
Importing pipe 52284-5517 discharge at 2022-12-19 14:07:26.437353
Importing pipe 52286-5519 discharge at 2022-12-19 14:07:26.445360
Importing pipe 52458-5640 discharge at 2022-12-19 14:07:26.455394
Importing pipe 52588-5748 discharge at 2022-12-19 14:07:26.464403
Importing pipe 52670-5813 discharge at 2022-12-19 14:07:26.474412
Importing pipe 52671-5814 discharge at 2022-12-19 14:07:26.482419
Importing pipe 52712-5849 discharge at 2022-12-19 14:07:26.491427
Importing pipe 52713-5850 discharge at 2022-12-19 14:07:26.499435
Importing pipe 52723-5857 discharge at 2022-12-19 14:07:26.508474
Importing pipe 53089-6110 discharge at 2022-12-19 14:07:26.518511
Importing pipe 53261-6239 discharge at 2022-12-19 14:07:26.528544
Importing pipe 53290-6268 discharge at 2022-12-19 14:07:26.537585
Importing 

Importing pipe 54877-6756 velocity at 2022-12-19 14:07:27.483266
Importing pipe 55884-6921 velocity at 2022-12-19 14:07:27.490273
Importing pipe 56103-6982 velocity at 2022-12-19 14:07:27.498280
Importing pipe 992-7041 velocity at 2022-12-19 14:07:27.505286
Importing pipe Link_1125-7071 velocity at 2022-12-19 14:07:27.512293
Importing pipe Link_120-7081 velocity at 2022-12-19 14:07:27.519299
Importing pipe Link_141_FSA_S-7103 velocity at 2022-12-19 14:07:27.525305
Importing pipe Link_165a-7121 velocity at 2022-12-19 14:07:27.532344
Importing pipe Link_167-7123 velocity at 2022-12-19 14:07:27.539349
Importing pipe Weir:11E-7224 velocity at 2022-12-19 14:07:27.547412
Importing pipe Weir:13W-7226 velocity at 2022-12-19 14:07:27.552459
Importing pipe Weir:15W-7227 velocity at 2022-12-19 14:07:27.562515
Importing pipe Weir:16W_New-7228 velocity at 2022-12-19 14:07:27.568519
Importing pipe Weir:17W-7229 velocity at 2022-12-19 14:07:27.574525
Importing pipe Weir:1F-7231 velocity at 2022-12-19

Importing pipe 52670-5813 discharge at 2022-12-19 14:07:33.238889
Importing pipe 52671-5814 discharge at 2022-12-19 14:07:33.250900
Importing pipe 52712-5849 discharge at 2022-12-19 14:07:33.262912
Importing pipe 52713-5850 discharge at 2022-12-19 14:07:33.274922
Importing pipe 52723-5857 discharge at 2022-12-19 14:07:33.285982
Importing pipe 53089-6110 discharge at 2022-12-19 14:07:33.300031
Importing pipe 53261-6239 discharge at 2022-12-19 14:07:33.314121
Importing pipe 53290-6268 discharge at 2022-12-19 14:07:33.326169
Importing pipe 53405-6367 discharge at 2022-12-19 14:07:33.338222
Importing pipe 53414-6377 discharge at 2022-12-19 14:07:33.350769
Importing pipe 53540-6484 discharge at 2022-12-19 14:07:33.362875
Importing pipe 53664-6541 discharge at 2022-12-19 14:07:33.372918
Importing pipe 53713-6581 discharge at 2022-12-19 14:07:33.382961
Importing pipe 53917-6597 discharge at 2022-12-19 14:07:33.394002
Importing pipe 53938-6614 discharge at 2022-12-19 14:07:33.404011
Importing 

Importing pipe Weir:4F-7245 velocity at 2022-12-19 14:07:34.756357
Importing pipe Weir:4S-7246 velocity at 2022-12-19 14:07:34.766366
Importing pipe Weir:52121 Port Coquitlam Weir-7247 velocity at 2022-12-19 14:07:34.774373
Importing pipe Weir:5BE-7249 velocity at 2022-12-19 14:07:34.783382
Importing pipe Weir:6E-7251 velocity at 2022-12-19 14:07:34.792390
Importing pipe Weir:8AW-7254 velocity at 2022-12-19 14:07:34.800397
Importing pipe Weir:Baynes Overflow-7259 velocity at 2022-12-19 14:07:34.809405
Importing pipe Weir:Cliff Ave N Intercept-7260 velocity at 2022-12-19 14:07:34.818414
Importing pipe Weir:Cliff Ave N Wet Well Overflow-7261 velocity at 2022-12-19 14:07:34.826421
Importing pipe Weir:marshend-7262 velocity at 2022-12-19 14:07:34.835472
Importing pipe Weir:Maryhill Overflow Weir-7263 velocity at 2022-12-19 14:07:34.844480
Importing pipe Weir:NS4-SSO-7265 velocity at 2022-12-19 14:07:34.853528
Importing pipe Weir:ROYAL AVEPS OUTFALL-7267 velocity at 2022-12-19 14:07:34.8615

Importing pipe Link_165a-7121 discharge at 2022-12-19 14:07:44.425471
Importing pipe Link_167-7123 discharge at 2022-12-19 14:07:44.441486
Importing pipe Weir:11E-7224 discharge at 2022-12-19 14:07:44.458501
Importing pipe Weir:13W-7226 discharge at 2022-12-19 14:07:44.473515
Importing pipe Weir:15W-7227 discharge at 2022-12-19 14:07:44.490530
Importing pipe Weir:16W_New-7228 discharge at 2022-12-19 14:07:44.506545
Importing pipe Weir:17W-7229 discharge at 2022-12-19 14:07:44.521591
Importing pipe Weir:1F-7231 discharge at 2022-12-19 14:07:44.537668
Importing pipe Weir:1G-7232 discharge at 2022-12-19 14:07:44.552740
Importing pipe Weir:1W-7233 discharge at 2022-12-19 14:07:44.567775
Importing pipe Weir:20th Street PS Overflow-7234 discharge at 2022-12-19 14:07:44.583840
Importing pipe Weir:2F-7236 discharge at 2022-12-19 14:07:44.598879
Importing pipe Weir:2G-7237 discharge at 2022-12-19 14:07:44.613942
Importing pipe Weir:2S-7238 discharge at 2022-12-19 14:07:44.628981
Importing pipe 

Importing pipe Weir:Short St-7268 velocity at 2022-12-19 14:07:46.159311
Importing pipe Weir:Slaughterhouse-7269 velocity at 2022-12-19 14:07:46.171322
Importing pipe Weir:SSO weir-7270 velocity at 2022-12-19 14:07:46.183333
Importing pipe Valve:Valve_2-7278 velocity at 2022-12-19 14:07:46.194343
Importing pipe Orifice:Braid Street Gate-7398 velocity at 2022-12-19 14:07:46.206384
Importing pipe Orifice:KatzieDuckBill-7403 velocity at 2022-12-19 14:07:46.217429
Importing pipe Orifice:Orifice_1-7408 velocity at 2022-12-19 14:07:46.228462


In [338]:
#PERMANENT CELL 9
#Create HTML reports

error_list = []

for gauge in gauges.index:
    if gauge in list(periods.Meter):

        location_type = gauges.loc[gauge,'Location Type']
        zone = str(gauges.loc[gauge,'Location'])
        pipe = str(gauges.loc[gauge,'Pipe'])
        node = str(gauges.loc[gauge,'Node1 (Or Pipe if pipe level)'])
        rain_gauge = str(gauges.loc[gauge,'Rain Gauge'])
        MonitoringType = str(gauges.loc[gauge,"Sensor Output"])
        NodeID = str(gauges.loc[gauge, "Node1 (Or Pipe if pipe level)"])
        MHName = str(gauges.loc[gauge,'Node1 AssetName']) 
        number_catchments = wwf_specs.loc[zone,'Number sanitary'] \
            + wwf_specs.loc[zone,'Number combined'] \
            + wwf_specs.loc[zone,'Number stormwater']
        
        effluent_types = []
        if wwf_specs.loc[zone,'Number sanitary'] > 0:
            effluent_types.append('Sanitary')
        if wwf_specs.loc[zone,'Number combined'] > 0:
            effluent_types.append('Combined')
        if wwf_specs.loc[zone,'Number stormwater'] > 0:
            effluent_types.append('Storm')
        
        for i, effluent_type in enumerate(effluent_types):
            if i == 0:
                effluent_type_string = effluent_type
            elif i == len(effluent_types)-1:
                effluent_type_string += ' and ' + effluent_type
            else:
                effluent_type_string += ', ' + effluent_type          
            
                    
        local_population = 0
        for ww_type in ww_types:
            if ww_type in res_types:                
                local_population += dwf_specs.loc[zone,ww_type + '_Population']
                
        
        total_area = round(wwf_specs.loc[zone,'Drainage area (ha)'],1)

        PipeShape = str(gauges.loc[gauge,'Pipe Shape'])
        try:
            PipeDimension = str(round(gauges.loc[gauge, 'Pipe Dimension'],2))
        except:
            PipeDimension = gauges.loc[gauge, 'Pipe Dimension']


        with open(output_folder + "\\" + "Calibration_Report_" + gauge + ".html", 'w') as f:
            
            f.write('<style>')
#             f.write('@page {margin-left: 5cm;}')
            f.write('p    {width:1000px;}')
            f.write('p    { font-size: 20px;}')
#---------------------------------------------------------------------------------------------------------------------        
 
    
            f.write('* {\n')
            f.write('  box-sizing: border-box;\n')
            f.write('}\n')

            f.write('.row {\n')
            f.write('  margin-left:0px;\n')
            f.write('  margin-right:-5px;\n')

            f.write('}\n')

            f.write('.column {\n')
            f.write('  float: left;\n')
            f.write('  width: 17%;\n')
            f.write('  padding: 5px;\n')
            f.write('}\n')

            f.write('/* Clearfix (clear floats) */\n')
            f.write('.row::after {\n')
            f.write('  content: "";\n')
            f.write('  clear: both;\n')
            f.write('  display: table;\n')
            f.write('}\n')

            f.write('table.first, table.first th, table.first td {\n')
            f.write('  border: 2px solid;\n')
            f.write('  padding: 4px;\n')

            f.write('  border-collapse: collapse;\n')
            f.write('  height: 35px;\n')
            f.write('  width: 800px;\n')
            f.write('}\n') 

            f.write('table.second {\n')
            f.write('  border: 2px solid;\n')
            f.write('  height: 35px;\n')
            f.write('  width: 310px;\n')
            f.write('  padding: 1px;\n')
            f.write('  margin: 1px;\n')

            f.write('})\n')    
    
#---------------------------------------------------------------------------------------------------------------------        
             
#             f.write('table.grid, th, td {')
#             f.write('  border: 1px solid;')
#             f.write('  padding: 4px;')
#             f.write('  border-collapse: collapse;')
            
            f.write('})')            
            f.write('</style>')
            f.write('<h1 style="text-align:center">Model Calibration Report for gauge ' 
                    + gauge + ' in Zone ' + zone + ', ' + model_area + '</h1>')

            f.write('<h2>1. Description of the Calibration Area</h2>')
            f.write('<table class="first">\n')
            f.write('<tr>\n')
            f.write('<td>Number of Catchments</td>\n')
            f.write('<td>'+ str(number_catchments) +'</td>\n')
            f.write('</tr>\n')
            f.write('<tr>\n')
            f.write('<td>Zone Area (Ha)</td>\n')
            f.write('<td>'+ str(total_area) +'</td>\n')
            f.write('</tr>\n')
            f.write('<tr>\n')
            f.write('<td>Local Residential Population</td>\n')
            f.write('<td>'+ str(int(local_population)) +'</td>\n')
            f.write('</tr>\n')
            f.write('<tr>\n')
            f.write('<td>Effluent Type</td>\n')
            f.write('<td>'+ effluent_type_string +'</td>\n')
            f.write('</tr>\n')            
            for ww_type in ww_types:
                if ww_type not in res_types:
                    f.write('<tr>\n')
                    f.write('<td>' + ww_type + ' Area (Ha)</td>\n')
#                     f.write('<td>'+ str(round(dwf_specs.loc[zone,ww_type + '_' + description],1)) +'</td>\n')
                    f.write('<td>'+ str(round(dwf_specs.loc[zone,ww_type + '_Area'],1)) +'</td>\n')
                    f.write('</tr>\n')
            f.write('</tr>\n')
            f.write('</table>')
            
            f.write('<h2>2. Description of the Sensor and Network</h2>')            
            f.write('<table class="first">\n')
            f.write('<tr>\n')
            f.write('<td>'+ 'Monitoring Type' +'</td>\n')
            f.write('<td>'+ MonitoringType +'</td>\n')
            f.write('</tr>\n')
            f.write('<tr>\n')
            f.write('<td>'+ 'Node ID' +'</td>\n')
            f.write('<td>'+ node +'</td>\n')
            f.write('</tr>\n')
            f.write('<tr>\n')
            f.write('<td>'+ 'Pipe ID' +'</td>\n')
            f.write('<td>'+ pipe +'</td>\n')
            f.write('</tr>\n')
            f.write('<tr>\n')
            f.write('<td>'+ 'Pipe Shape' +'</td>\n')
            f.write('<td>'+ PipeShape +'</td>\n')
            f.write('</tr>\n')
            f.write('</tr>\n')
            f.write('<tr>\n')
            f.write('<td>'+ 'Pipe Dimension (m) or CRS ID ' +'</td>\n')
            f.write('<td>'+ PipeDimension +'</td>\n')
            f.write('</tr>\n')
            f.write('</table>')
    
            
            f.write('<h2>3. Model Calibration Setup: </h2>')
                    
            f.write('<table class="first">\n')
            f.write('  <tr>\n')
            f.write('    <th>Simulation</th>\n')
            f.write('    <th>Start Date</th>\n')
            f.write('    <th>End Date</th>\n')
            f.write('    <th>Duration</th>\n')
            f.write('  </tr>\n')

            for period_spec in period_specs:
                description = period_spec[:2] + ' Calibration'
                start = periods[periods.Meter==gauge][period_spec + ' Start'].iloc[0]
                end = periods[periods.Meter==gauge][period_spec + ' End'].iloc[0]
                if pd.isnull(start):
                    break
                duration = str((end - start).days) + ' days'
                
                f.write('  <tr>\n')
                f.write('    <td>' + description + '</th>\n')
                f.write('    <td>' + start.strftime("%Y-%m-%d %H:%M") + '</th>\n')
                f.write('    <td>' + end.strftime("%Y-%m-%d %H:%M") + '</th>\n')
                f.write('    <td>' + duration + '</th>\n')
                f.write('  </tr>\n')                   
            f.write('</table>\n')
                                      
            f.write('<h2>4. Zone Map</h2>\n')
            f.write('<img src="' + map_folder + '\\' + zone + '.jpg" alt="' + gauge + '">\n')
            f.write('<h2>5. DWF Calibration Summary</h2>\n')  
            f.write('<p>' + str(list(report_text[report_text.Zone==zone]['DWF Text'])[0]) + '</p>\n')
            f.write('<h2>6. WWF Calibration and Validation Summary</h2>\n')
            f.write('<p>' + str(list(report_text[report_text.Zone==zone]['WWF Text'])[0]) + '</p>\n')
            f.write('<h2>7. Calibration Issues</h2>\n')
            f.write('<p>' + str(list(report_text[report_text.Zone==zone]['Issue Text'])[0]) + '</p>\n')
            f.write('<h2>8. Recommendations</h2>\n')
            f.write('<p>' + str(list(report_text[report_text.Zone==zone]['Recommendation Text'])[0]) + '</p>\n')
            f.write('<h2>9. DWF Calibration Plots and Statistics</h2>\n')

            wwf_plot_header_added = False
            for period_spec in period_specs:
                
                start = periods[periods.Meter==gauge][period_spec + ' Start'].iloc[0]
                end = periods[periods.Meter==gauge][period_spec + ' End'].iloc[0]
                
                if pd.isnull(start):
                    break

                if period_spec[:3] == "WWF" and wwf_plot_header_added == False:
                    f.write('<h2>10. WWF Calibration Plots</h2>\n')
                    wwf_plot_header_added = True                                                

                chart_header = period_spec[:3] + ', ' + start.strftime("%b %d %Y") + ' - ' + end.strftime("%b %d %Y")

                measured_gauge = measured.loc[(measured.Gauge==gauge) & (measured.index >= start) & (measured.index <= end)].copy()
                
                if location_type == 'PS':
                    measured_gauge['Discharge_Hourly'] = measured_gauge['Flow'].rolling('1h').mean()
                
                measured_rain = rainfall.loc[(rainfall.index >= start) & (rainfall.index <= end)][rain_gauge].to_frame()

                result_flow_gauge = flow_df_all.loc[(flow_df_all.MUID==pipe) & (flow_df_all.index >= start) & (flow_df_all.index <= end)].copy()
                
                result_velocity_gauge = velocity_df_all.loc[(velocity_df_all.MUID==pipe) & (velocity_df_all.index >= start) & (velocity_df_all.index <= end)].copy()
                
                spill_df_zone = spill_df_zones.loc[(spill_df_zones.Zone==zone) & (spill_df_zones.Date_Time >= start) & (spill_df_zones.Date_Time <= end)].copy()                
                spill_df_all_zone = spill_df_all.loc[(spill_df_all.Zone==zone) & (spill_df_all.index >= start) & (spill_df_all.index <= end)].copy()
                spill_df_all_zone.drop(columns=['Zone'],inplace=True)
                spill_top_ten = spill_df_all_zone.groupby(['Node']).max()
                spill_top_ten.reset_index(inplace=True)
                spill_top_ten.sort_values(by='Spill',ascending=False,inplace=True)
                spill_top_ten.reset_index(drop=True,inplace=True)
                spill_top_ten = spill_top_ten[spill_top_ten['Spill']>0]
                spill_top_ten = spill_top_ten.head(10)                                
                
                if location_type == 'PS':
                    result_flow_gauge['Discharge_Hourly'] = result_flow_gauge['Discharge'].rolling('1h').mean()
                
                result_level_gauge = level_df_all.loc[(level_df_all.MUID==node) & (level_df_all.index >= start) & (level_df_all.index <= end)]

                has_outfalls = False
                if zone in list(outfalls.Zone):
                    has_outfalls = True
                    outfalls_filter = outfalls[outfalls.Zone==zone].copy()
                    outfalls_filter.reset_index(inplace=True)

                    for index, row in outfalls_filter.iterrows():
                        overflow_pipe = row['Res_ID']
                        result_outfalls = flow_df_all.loc[(flow_df_all.MUID==overflow_pipe) & (flow_df_all.index >= start) & (flow_df_all.index <= end)].copy()
                        result_outfalls['Outfall'] = row['Outfall']
                        if index == 0:
                            result_outfalls_all = result_outfalls.copy()
                        else:
                            result_outfalls_all = pd.concat([result_outfalls_all,result_outfalls])
                
                
            #     result_gauge = df_result.loc[(df_result.index >= start) & (df_result.index <= end)]
                #---------------------------------------------------------------------------------------------
                compare_stats = []
                peak_level_model = round(result_level_gauge.Level.max(),2)
                compare_stats.append(['Peak Level Model',peak_level_model,'m'])
                peak_level_measured = round(measured_gauge.Level.max(),2)
                compare_stats.append(['Peak Level Measured',peak_level_measured,'m'])
                compare_stats.append(['Peak Level Difference',round(peak_level_model - peak_level_measured,2),'m'])
                volume_model = round(result_flow_gauge.Volume.sum(),0).item()
                compare_stats.append(['Volume Model',volume_model,'m3'])
                volume_measured = round(measured_gauge.Volume.sum(),0).item() #convert to native float to avoid inf
                compare_stats.append(['Volume Measured',volume_measured,'m3'])
                try:
                    compare_stats.append(['Volume Difference',round((volume_model - volume_measured) / volume_measured * 100, 0),'%'])
                except:
                    compare_stats.append(['Volume Difference','','%'])
                peak_flow_model = round(result_flow_gauge.Discharge.max(),1)
                compare_stats.append(['Peak Flow Model',peak_flow_model,'L/s'])
                peak_flow_measured = round(measured_gauge.Flow.max(),1)
                compare_stats.append(['Peak Flow Measured',peak_flow_measured,'L/s'])
                try:
                    compare_stats.append(['Peak Flow Difference',round((peak_flow_model - peak_flow_measured) / peak_flow_measured * 100, 0),'%'])
                except:
                    compare_stats.append(['Peak Flow Difference','','%'])
                
                
                f.write('<h3>' + chart_header + '</h3>\n')  
                #----------------------------------------------------------------------------------------------------   
                if period_spec[:3] == "DWF":
                    
                    f.write('<div class="row">')
                    
                    f.write('<div class="column">')    
                    f.write('<table class="second">\n')
                    f.write('  <tr>\n')
                    f.write('    <th>Description</th>\n')
                    f.write('    <th>Value</th>\n')
                    f.write('    <th>Unit</th>\n')
                    f.write('  </tr>\n')                   
                    
                    #sim and measured stats
                    for compare_stat in compare_stats:
                        f.write('  <tr>\n')
                        f.write('    <td>' + compare_stat[0] + '</th>\n')
                        f.write('    <td>' + str(compare_stat[1]) + '</th>\n')
                        f.write('    <td>' + compare_stat[2] + '</th>\n')
                        f.write('  </tr>\n')
                        
                    f.write('</table>\n')
                    f.write('</div>')                    
                    
                    #Loading rate
                    f.write('<div class="column">')    
                    f.write('<table class="second">\n')
                    f.write('  <tr>\n')
                    f.write('    <th>Load Type</th>\n')
                    f.write('    <th>Loading Rate</th>\n')
                    f.write('    <th>Unit</th>\n')
                    f.write('  </tr>\n')
                                       
                    for ww_type in ww_types:
                        if ww_type in res_types:
                            decimals = 3
                        else: 
                            decimals = 1
                        rate = round(dwf_specs.loc[zone,ww_type + '_Rate'],decimals)
                        unit = all_types.loc[ww_type,'Unit2']

                        f.write('  <tr>\n')
                        f.write('    <td>' + ww_type + '</th>\n')
                        f.write('    <td>' + str(rate) + '</th>\n')
                        f.write('    <td>' + unit + '</th>\n')
                        f.write('  </tr>\n') 
                    f.write('<tr><td><br/></td><td><br/></td><td><br/></td></tr>\n')
                    f.write('<tr><td><br/></td><td><br/></td><td><br/></td></tr>\n')
                    f.write('<tr><td><br/></td><td><br/></td><td><br/></td></tr>\n')
                    f.write('</table>\n')
                    f.write('</div>')

                    f.write('<div class="column">')

                    #Population and ICI areas
                    f.write('<table class="second">\n')
                    f.write('  <tr>\n')
                    f.write('    <th>Load Type</th>\n')
                    f.write('    <th>Local</th>\n')
                    if use_accumulation == True:
                        f.write('    <th>Total</th>\n')
                    else:
                        f.write('    <th></th>\n')
                    f.write('    <th>Unit</th>\n')
                    f.write('  </tr>\n')

                    for ww_type in ww_types:
                        if ww_type in res_types:
                            decimals = 0
                        else: 
                            decimals = 1
                        description = unit = all_types.loc[ww_type,'Description']
                        local_val = round(dwf_specs.loc[zone,ww_type + '_' + description],decimals)
                        if use_accumulation == True:
                            total_val = round(dwf_specs.loc[zone,ww_type + '_' + description + '_Upstream'],decimals)
                        unit = all_types.loc[ww_type,'Unit1']

                        f.write('  <tr>\n')
                        f.write('    <td>' + ww_type + '</th>\n')
                        f.write('    <td>' + str(local_val) + '</th>\n')
                        if use_accumulation == True:
                            f.write('    <td>' + str(total_val) + '</th>\n')
                        else:
                            f.write('    <td></th>\n')
                        f.write('    <td>' + unit + '</th>\n')
                        f.write('  </tr>\n') 
                    f.write('<tr><td><br/></td><td><br/></td><td><br/></td><td><br/></td></tr>\n')
                    f.write('<tr><td><br/></td><td><br/></td><td><br/></td><td><br/></td></tr>\n')
                    f.write('<tr><td><br/></td><td><br/></td><td><br/></td></tr>\n')
                    f.write('</table>\n')
                    f.write('</div>')

                    f.write('<div class="column">')
                    #Waterload
                    f.write('<table class="second">\n')
                    f.write('  <tr>\n')
                    f.write('    <th>Load Type</th>\n')
                    f.write('    <th>Local</th>\n')
                    if use_accumulation == True:
                        f.write('    <th>Total</th>\n')
                    else:
                        f.write('    <th></th>\n')
                    f.write('    <th>Unit</th>\n')
                    f.write('  </tr>\n')

                    for load_type in all_types.index.values.tolist():

                        local_val = round(dwf_specs.loc[zone,load_type + '_WaterLoad'],0)
                        if use_accumulation == True:
                            total_val = round(dwf_specs.loc[zone,load_type + '_WaterLoad_Upstream'],0)
                        unit = 'm3/d'

                        f.write('  <tr>\n')
                        f.write('    <td>' + load_type + '</th>\n')
                        f.write('    <td>' + str(local_val) + '</th>\n')
                        if use_accumulation == True:
                            f.write('    <td>' + str(total_val) + '</th>\n')
                        else:
                            f.write('    <td></th>\n')
                        f.write('    <td>' + unit + '</th>\n')
                        f.write('  </tr>\n') 
                        
                    f.write('<tr><td><br/></td><td><br/></td><td><br/></td></tr>\n')

                    f.write('</table>\n')
                    f.write('</div>')
#                     f.write('</div>')
                    
                else:

                    total_rows = 10
                    f.write('<div class="row">')
                    for i in range(0,4): #Cycle through the three table types.   
                        f.write('<div class="column">')    
                        f.write('<table class="second">\n')
                        f.write('  <tr>\n')
                        f.write('    <th>Description</th>\n')
                        f.write('    <th>Value</th>\n')
                        f.write('    <th>Unit</th>\n')
                        f.write('  </tr>\n')

                        current_row = 0
                        if i == 0:
                            for compare_stat in compare_stats:
                                f.write('  <tr>\n')
                                f.write('    <td>' + compare_stat[0] + '</th>\n')
                                f.write('    <td>' + str(compare_stat[1]) + '</th>\n')
                                f.write('    <td>' + compare_stat[2] + '</th>\n')
                                f.write('  </tr>\n')
                                current_row += 1                
                        else:
                            for index, row in wwf_stats_specs[wwf_stats_specs.Table==i].iterrows():

                                decimals = row['Decimals']

#                                 rate = round(dwf_specs.loc[zone,ww_type + '_Rate'],decimals)
#                                 unit = all_types.loc[ww_type,'Unit2']


                                f.write('  <tr>\n')
                                f.write('    <td>' + row['Description'] + '</th>\n')
                                if row['DF'] == 'wwf_specs':
                                    if slope_source_unit_meter_per_meter == False and row['Description'] == 'Average slope':
                                        f.write('    <td>' + str(round(wwf_specs.loc[zone,row['DF_Col']]/1000,decimals)) + '</th>\n')
                                    elif decimals > 0:
                                        f.write('    <td>' + str(round(wwf_specs.loc[zone,row['DF_Col']],decimals)) + '</th>\n')
                                    else:
                                        f.write('    <td>' + str(int(wwf_specs.loc[zone,row['DF_Col']])) + '</th>\n')
                                else: 
                                    f.write('    <td>' + str(round(measured_rain[rain_gauge].sum(),decimals)) + '</th>\n')
                                f.write('    <td>' + row['Unit'] + '</th>\n')
                                f.write('  </tr>\n')
                                current_row += 1

                        remaining_rows = total_rows - current_row

                        for remaining_row in range(remaining_rows):
                            f.write('<tr><td><br/></td><td><br/></td><td><br/></td></tr>\n')
                        f.write('</table>\n')
                        f.write('</div>')
                        
                #Spill top 10
                if period_spec[:3] == "DWF":
                    total_rows = 9
                else:
                    total_rows = 10
                    
                f.write('<div class="row">')
                  
                f.write('<div class="column">')    
                f.write('<table class="second">\n')
                f.write('  <tr>\n')
                f.write('    <th>Node Spill Top 10</th>\n')
                f.write('    <th>Peak Spill (L/s)</th>\n')
                f.write('  </tr>\n')

                current_row = 0

                for index, row in spill_top_ten.iterrows():
                    f.write('  <tr>\n')
                    f.write('    <td>' + row['Node'] + '</th>\n')
                    f.write('    <td>' + str(int(row['Spill'])) + '</th>\n')
                    f.write('  </tr>\n')
                    current_row += 1                


                remaining_rows = total_rows - current_row

                for remaining_row in range(remaining_rows):
                    f.write('<tr><td><br/></td><td><br/></td></tr>\n')
                f.write('</table>\n')
                f.write('</div>')
        

                f.write('</div>')
                    #----------------------------------------------------------------------------------------------


                fig = plotly.subplots.make_subplots(rows=1,cols=2,subplot_titles=(
                                                    'Discharge at Gauge ' + gauge + ', Reach ' + pipe + ', Rain Gauge ' + rain_gauge,  
                                                    'Water Level at Gauge ' + gauge + ', Node ' + node),
                                                    specs=[[{"secondary_y": True}, {"secondary_y": True}]],
                                                   horizontal_spacing = 0.1)

                fig.add_trace(go.Scatter(x=result_flow_gauge.DateTimeRef, y=result_flow_gauge.Discharge, mode='lines',name='Model Flow'),1,1)   
                fig.add_trace(go.Scatter(x=measured_gauge.index, y=measured_gauge.Flow, mode='lines',name='Measured Flow'),1,1)
                fig.add_trace(go.Scatter(x=measured_rain.index, y=measured_rain[rain_gauge], mode='lines',name='Rainfall'),1,1, secondary_y=True)

                if location_type == 'PS':
                    fig.add_trace(go.Scatter(x=result_flow_gauge.DateTimeRef, y=result_flow_gauge.Discharge_Hourly, mode='lines',name='Model Flow Hourly',line_color='yellow'),1,1)   
                    fig.add_trace(go.Scatter(x=measured_gauge.index, y=measured_gauge.Discharge_Hourly, mode='lines',name='Measured Flow Hourly',line_color='black'),1,1)
                
                fig.add_trace(go.Scatter(x=result_level_gauge.index, y=result_level_gauge.Level, mode='lines',name='Model Level',line_color='green'),1,2)
                fig.add_trace(go.Scatter(x=measured_gauge.index, y=measured_gauge.Level, mode='lines',name='Measured Level',line_color='purple'),1,2)
                           
                fig.update_layout(
                    autosize=False,
                    width = 1600,
                    height=300,
                    margin=dict(
                        l=0,
                        r=0,
                        b=25,
                        t=60,
                        pad=4
                        ),
                    title ={
                        'text' : chart_header,
                        'x':0.45,
                        'xanchor': 'center'
                    })
                
                fig['layout']['yaxis']['title']='Dicharge (L/s)'
                fig['layout']['yaxis2']['title']='Rainfall (mm / 5 min)' 
                fig['layout']['yaxis2']['range']=[10,0]
                fig['layout']['yaxis3']['title']='Water Level (m)'
                
                f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
                f.write('<br>')
                
                fig = plotly.subplots.make_subplots(rows=1,cols=2,subplot_titles=(
                                                    'Velocity at Gauge ' + gauge + ', Reach ' + pipe,
                                                    'Total Manhole Spills, CSO and SSO'),
                                                    specs=[[{"secondary_y": False}, {"secondary_y": False}]],
                                                   horizontal_spacing = 0.1)
                
                fig.add_trace(go.Scatter(x=result_velocity_gauge.DateTimeRef, y=result_velocity_gauge.Velocity, mode='lines',name='Model Velocity'),1,1)   
                fig.add_trace(go.Scatter(x=measured_gauge.index, y=measured_gauge.Velocity, mode='lines',name='Measured Velocity'),1,1)
                
                spill_df_zone
                fig.add_trace(go.Scatter(x=spill_df_zone.Date_Time, y=spill_df_zone.Spill, mode='lines',name='Total Manhole Spilling'),1,2)
                if has_outfalls == True:
                
                    show_first = []
                    show_last = []
                    for outfall in list(outfalls_filter.Outfall):
                        if outfall in measured.Gauge.unique():
                            show_first.append(outfall)
                        else:
                            show_last.append(outfall)
                    outfalls_to_plot = show_first + show_last
                    for i, outfall in enumerate(outfalls_to_plot):
                        result_outfalls_all_filter = result_outfalls_all[result_outfalls_all.Outfall==outfall]
                        if i < len(colors):
                            fig.add_trace(go.Scatter(x=result_outfalls_all_filter.index, 
                                                     y = result_outfalls_all_filter.Discharge, 
                                                     mode='lines',name=outfall,line={'color': colors[i]}),1,2)
                        else:
                            fig.add_trace(go.Scatter(x=result_outfalls_all_filter.index, 
                                                     y = result_outfalls_all_filter.Discharge, 
                                                     mode='lines',name=outfall),1,2)
                         
                        if outfall in measured.Gauge.unique():
                            outfall_measured = measured.loc[(measured.Gauge==outfall) & (measured.index >= start) & (measured.index <= end)].copy()    
                            if i < len(colors):
                                fig.add_trace(go.Scatter(x=outfall_measured.index, 
                                                         y=outfall_measured.Flow, 
                                                         mode='lines',name=outfall + ' - Measured',
                                                         line={'color': colors[i],'dash': 'dash'}),1,2)                                                         
                            else:
                                fig.add_trace(go.Scatter(x=outfall_measured.index, 
                                                         y=outfall_measured.Flow, 
                                                         mode='lines',name=outfall + ' - Measured',
                                                         line={'dash': 'dash'}),1,2)
                                                                                         
                
                fig.update_layout(
                    autosize=False,
                    width = 1600,
                    height=300,
                    margin=dict(
                        l=0,
                        r=0,
                        b=25,
                        t=60,
                        pad=4
                        ))
#                     title ={
#                         'text' : chart_header,
#                         'x':0.45,
#                         'xanchor': 'center'
#                     })
                
                fig['layout']['yaxis']['title']='Velocity (m/s)'
                fig['layout']['yaxis2']['title']='Dicharge (L/s)'
                
                f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
                f.write('<br>')
                          

        f.close()
        print("Writing html for " + gauge)
#     except Exception as e:
#         error_list.append([gauge_id,e])

    

Writing html for 20th_Street_PS
Writing html for AL1
Writing html for AL3
Writing html for AN1
Writing html for BI1
Writing html for BIN18
Writing html for BN17
Writing html for BN21
Writing html for BNC9
Writing html for CQ1
Writing html for CQ20
Writing html for FST6
Writing html for GLT7A
Writing html for HT4
Writing html for LKI55
Writing html for LZ20
Writing html for LZ4
Writing html for LZG32
Writing html for MH20
Writing html for ML1
Writing html for ML2
Writing html for ML1-2
Writing html for ML7
Writing html for NC1
Writing html for NR4B
Writing html for NRT20
Writing html for NW22
Writing html for NW25
Writing html for NW2A
Writing html for NW2B
Writing html for NW2
Writing html for NW3
Writing html for NW5
Writing html for NW70
Writing html for NW71
Writing html for NW72
Writing html for NW73
Writing html for NWP6
Writing html for PM101
Writing html for PM3
Writing html for PM30
Writing html for POC8D
Writing html for Port_Coquitlam_PS
Writing html for Port_Moody_PS
Writing

In [ ]:
#PERMANENT CELL 10
#Create csv files for confidence maps.

if generate_confidence_csvs == True:

    color_dict = {}
    color_dict[1] = 'Green'
    color_dict[1.5] = 'Light Green'
    color_dict[2] = 'Yellow'
    color_dict[2.5] = 'Light Red'
    color_dict[3] = 'Red'


    event_categories = ['DWF Calibration','WWF Calibration','WWF Validation']

    result_types = ['HGL','Flow','Volume']

    result_points = []
    zone_review = []

    for index, row in map_periods.iterrows():
        gauge = row['Meter']
        zone = row['Zone']
        meter_status = row['Meter Status']
        x = gauges.loc[gauge,'X coordinate']
        y = gauges.loc[gauge,'Y coordinate']
        x_shift = gauges.loc[gauge,'Shift X (m)']
        y_shift = gauges.loc[gauge,'Shift Y (m)']
        catchment_type = gauges.loc[gauge,'Catchment Type']

        print('Processing ' + gauge + ' for confidence maps.')

        for i, event_category in enumerate(event_categories):
            event_category_short = event_category[:7]
            start = row[event_category_short + ' Start']
            end = row[event_category_short + ' End']
            y_minor_shift = -i * map_point_spacing
            y_final = y + y_shift + y_minor_shift

            color_values = []
            for j, result_type in enumerate(result_types):
                x_minor_shift = j * map_point_spacing
                x_final = x + x_shift + x_minor_shift
                bad_data = False
                if result_type == 'HGL':
                    result_type_csv = 'Max HGL'
                    qa = row[event_category_short + ' WL QA']
                    if qa.lower() == 'bad data':
                        bad_data = True
                    else:
                        node = str(gauges.loc[gauge,'Node1 (Or Pipe if pipe level)'])
                        max_model = level_df_all.loc[(level_df_all.MUID==node) & (level_df_all.index >= start) & (level_df_all.index <= end)].Level.max()
                        max_measured = measured.loc[(measured.Gauge==gauge) & (measured.index >= start) & (measured.index <= end)].Level.max()
                        if not math.isnan(max_measured) and max_measured != 0:
                            diff = abs(max_model - max_measured) * 100 #cm
                        else:
                            diff = 'NA'

                elif result_type == 'Flow':
                    result_type_csv = 'Max Flow'
                    qa = row[event_category_short + ' Flow QA']
                    if qa.lower() == 'bad data':
                        bad_data = True
                    else:
                        pipe = str(gauges.loc[gauge,'Pipe'])
                        filter_model = flow_df_all.loc[(flow_df_all.MUID==pipe) & (flow_df_all.index >= start) & (flow_df_all.index <= end)].copy()
                        filter_model['Hourly'] = filter_model.Discharge.rolling('1h').mean()
                        filter_model = filter_model.iloc[12: , :] #Remove first hour to not get skewed hourly average there                        
                        max_model = filter_model.Hourly.max()

                        filter_measured = measured.loc[(measured.Gauge==gauge) & (measured.index >= start) & (measured.index <= end)].copy()
                        filter_measured['Hourly'] = filter_measured.Flow.rolling('1h').mean()
                        filter_measured = filter_measured.iloc[12: , :] #Remove first hour to not get skewed hourly average there 
                        max_measured = filter_measured.Hourly.max()
                        if not math.isnan(max_measured) and max_measured != 0:
                            diff = abs(max_model - max_measured) / max_measured
                        else:
                            diff = 'NA'

                else:
                    result_type_csv = 'Acc Volume'
                    qa = row[event_category_short + ' Flow QA']
                    if qa.lower() == 'bad data':
                        bad_data = True
                    else:
                        pipe = str(gauges.loc[gauge,'Pipe'])
                        max_model = flow_df_all.loc[(flow_df_all.MUID==pipe) & (flow_df_all.index >= start) & (flow_df_all.index <= end)].Volume.sum()
                        max_measured = measured.loc[(measured.Gauge==gauge) & (measured.index >= start) & (measured.index <= end)].Volume.sum()
                        if not math.isnan(max_measured) and max_measured != 0:
                            diff = abs(max_model - max_measured) / max_measured
                        else:
                            diff = 'NA'

                if bad_data == True:
                    color = 'Black'
                    color_value = 0
                elif diff == 'NA':
                    color = 'Grey'
                    color_value = 0
                else:
                    if event_category[:3] == 'WWF':
                        threshold_column = event_category[:3] + ' ' + catchment_type + ' ' + result_type + ' Status'
                    else:
                        threshold_column = event_category[:3] + ' ' + result_type + ' Status'
                    for k in range(3):
                        threshold = thresholds.loc[k,threshold_column]
                        if diff >= threshold:
                            color = thresholds.loc[k,'Color']
                            color_value = thresholds.loc[k,'index']
                            break

                color_values.append(color_value)

                label = ''
                if i == 0 and j == 0:
                    label = gauge

                result_points.append([gauge,x_final,y_final,label,event_category,result_type_csv,color])

            if meter_status == 'Primary':
                weighting_components = []
                weighted_color_value = 0
                value_count = 0
                qa = 'OK'
                for k, color_value in enumerate(color_values):
                    if color_value > 0:
                        if k == 0:
                            weighting_components.append(color_value) #Level is weighted as 50%
                            value_count += 1
                        else:
                            weighting_components.append(color_value/2) #Flow is weighted as 25%
                            value_count += 0.5
                    else:
                        if k == 0:
                            qa = 'Level Issue'
                        else:
                            qa = 'Flow Issue'                       

                if len(weighting_components) == 0:
                    weighted_color_value = 0
                    weighted_color = ''
                    qa = 'All Data Issue'
                else:
                    sum_color_values = 0
                    for weighting_component in weighting_components:
                        sum_color_values += weighting_component
                    weighted_color_value = sum_color_values / value_count
                    weighted_color_value = round((weighted_color_value+0.001)*2,0)/2 #2.25 should round up to 2.5 as in Excel
                    weighted_color = color_dict[weighted_color_value]
                zone_review.append([event_category,zone,weighted_color,qa])

    result_points = pd.DataFrame(result_points,columns=['Gauge','X','Y','Label','SimType','ResultType','Color'])
    result_points.to_csv(output_folder + '\\ResultPoints.csv',index=False)

    zone_review = pd.DataFrame(zone_review,columns=['Event','Location','Status','QA'])
    for event_category in event_categories:
        event_category_short = event_category[:7]
        event_category_short = event_category_short.replace(' ','_')
        if event_category_short == 'DWF_Cal':
            event_category_short = 'DWF'
        zone_review[zone_review.Event==event_category][['Location','Status','QA']].to_csv(output_folder + '\\Zone_Review_' + event_category_short +'.csv',index=False)




In [347]:
measured_stats = measured.copy()
measured_stats['Start'] = measured.index
measured_stats['End'] = measured.index
measured_stats = measured_stats[['Gauge','Start','End']]
measured_stats = measured_stats.groupby(['Gauge']).agg({'Start':'min','End':'max'})
measured_stats.reset_index(inplace=True)
measured_stats.to_csv(r"J:\SEWER_AREA_MODELS\FSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Version_43\FSA_Calibration_Specs\Sim_Periods.csv",index=False)
